In [43]:
# Put these at the top of every notebook to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# General imports
from fastai.imports import *
# Data augmentation (transformations)
from fastai.transforms import * 
# Convolutional learner class
from fastai.conv_learner import *
# Model architecture
from fastai.model import *
# Dataset handling class
from fastai.dataset import *
# SGD with restarts
from fastai.sgdr import *
# Plotting
from fastai.plots import *

# Check GPU hardware is available
print(f'Cuda available: {torch.cuda.is_available()}')
print(f'Cudnn enabled:  {torch.backends.cudnn.enabled}')

PATH = "/home/paperspace/data/dogscats/"

# Uncomment the below if you need to reset your precomputed activations
# shutil.rmtree(f'{PATH}tmp', ignore_errors=True)

sz = 224
arch = resnet34

Cuda available: True
Cudnn enabled:  True


In [44]:
tfms = tfms_from_model(f_model=arch, sz=sz) # this is has no transformation
data = ImageClassifierData.from_paths(path=PATH, tfms=tfms)
# learn = ConvLearner.pretrained(f=arch, data=data, precompute=True)

log_preds = np.load('log_preds.npy', allow_pickle=False)

In [ ]:
def most_by_mask(mask, mult):
    """called by most_by_correct
    returns an array of image IDs given a boolean matrix"""
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct):
    """returns an array of image IDs"""
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)


def load_img_id(ds, idx):
    """called by plot_val_with_title
    load a pic & return it as a np.array"""
    return np.array(PIL.Image.open(PATH+ds.fnames[idx]))

def plots(ims, figsize=(12,6), rows=1, titles=None):
    """called by plot_val_with_title
    create a subplot of pics"""
    f = plt.figure(figsize=figsize) # figsize: Width, Height in inches
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1) # add_subplot(nrows, ncols, index)
        sp.axis('Off')
        if titles is not None: sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i])

def plot_val_with_title(idxs, title):
    """take a list of image indicies and plot them on a row"""
    imgs = [load_img_id(data.val_ds,x) for x in idxs]# a list of images
    title_probs = [probs[x] for x in idxs]# a list probabilities
    print(title)
    return plots(imgs, rows=1, titles=title_probs, figsize=(16,8)) if len(imgs)>0 else print('Not Found.')


### Objective

Dissect and recreate these:

```python
def most_by_mask(mask, mult):
    """called by most_by_correct
    returns an array of image IDs given a boolean matrix"""
    idxs = np.where(mask)[0]
    return idxs[np.argsort(mult * probs[idxs])[:4]]

def most_by_correct(y, is_correct):
    """returns an array of image IDs"""
    mult = -1 if (y==1)==is_correct else 1
    return most_by_mask(((preds == data.val_y)==is_correct) & (data.val_y == y), mult)

```

---

Practice topics:
- np.where
- np.exp
- np.argmax
- np.where
- get a filter/mask of the correctly predicted array
- get the indicies of correctly predicted rows


---

### Assignment

Given these variables:
- `log_preds` : a (2000, 2) array of log probabilities for cats & dogs
- `data.val_y` : an (2000,) array of 1s and 0s

1. Find the row indicies that are _most_ correct.
1. Find the row indicies that are _most_ incorrect
1. Plot 8 of the _most_ incorrect images

Recall that `log_preds` is returned by `learn.predict()`. This is the result we have to work with.

---